# CS-6570 Lecture 12 - Classification, Logistic Regression, and Gradient Descent
**Instructor: Dylan Zwick**

*Weber State University*

Today's lecture will be a shorter one, to make time to review the regression material before the exam. However, I want to give you a glimpse of the next big topic we'll be covering - classification. We'll look at this in the context on a simple dataset of fruit measurements.

First, as usual, we'll start by importing our favorite libraries. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Classification Basics

For our initial discussion of classification, we'll use a simple dataset of measurements from fruits:

In [ ]:
fruits = pd.read_table('YOUR PATH HERE')
fruits.head()

It appears there are seven data columns here and 59 rows.

In [ ]:
print(fruits.shape)

There are four unique fruit names:

In [ ]:
print(fruits['fruit_name'].unique())

Suppose we want to predict the fruit (based on fruit name) using the other inputs like mass, width, height, and so on. Could we do this with regression?

Well, maybe, but it would be a bit awkward. With regression the goal is to predict a numerical response. Fruits aren't numbers. We could try and encode them numerically (as has been done with the fruit label), but viewing these as actual numbers and not labels is a bit silly. It would imply that, in some way, a lemon is four times an apple, and the difference between an apple and a mandarin is half the distance between a mandarin and a lemon. It just doesn't make a lot of sense.

Also, in regression the predicted outcomes are numbers, and so the model could, for example, predict a 5.2 or a -1.8. What fruit would that be?

Now, if there were only two categories in the classification, then you could attempt to use a regression model and pick a cutoff, so that anything above the cutoff goes to one category, and everything below goes to the other. This can work well enough when there are only two categories (in fact, it's essentially how linear discriminant analysis works with only two categories), but for more than two categories the approach falls apart. We need another way.

Returning to our dataset, let's take a look at how we might want to approach this classification. If we look at the scatterplot of our predictor variables against each other, what we'd hope to see is a clustering of the categories we would like to predict, in such a way that we can separate them out. Now, usually with real world data you don't get a perfect quarantine of your categories, but if we can find input variables that seem to provide some separation, that's very valuable for prediction.

In [ ]:
feature_names = ['mass', 'width', 'height', 'color_score']
X = fruits[feature_names]
y = fruits['fruit_label']

pd.plotting.scatter_matrix(X, c = y, marker = 'o', figsize=(15,15))
plt.show()

When we're doing classification, we want to find a set of variables that, if possible, separate our classes. It look like we might be able to do something like that using mass and height.

## Logistic Regression

<center>
    <img src="Log Regression.jpg" width="600">
</center>

Suppose I’m an instructor for a data science class, and I want to convince my students that they should
study. Of course, there are plenty of excellent reasons for studying, including many that will hopefully
be more persuasive than this one, but one matter-of-fact reason is that studying improves the liklihood of
passing the final exam. Say I wanted to look into this and understand it a little better. I want to understand
how many hours a student should study in order to pass the final exam, and how the probability of passing
the exam improved with the number of study hours. How can I do this? Well, one way is to use logistic
regression.

In today’s class we will cover:

- The fundamental concepts behind a logistic regression model, and how the “best” model for a set of
data can be determined.

- How to use logistic regression in Python with the sklearn library.

- Methods for evaluating the quality on a logistic regression model, including understanding the confusion matrix.

### Concepts and Theory

Today we’re going to focus on the simplest type of logistic regression, called binary logistic regression, which
is used when you want to predict a binary outcome like yes/no or true/false. There are more complicated
types of logistic regression like multinomial logistic regression, where there are more than two possible categories, or ordinal logistic regression, where there are more than two categories, and these categories have some natural ordering. 

For example, a multinomial logistic regression could look at hand written digits, and classify which digit, from 0 to 9, is written, while an ordinal logistic regression could look at the text of Yelp reviews and attempt to predict the number of stars (1 to 5) awarded for the review. While multinomial and ordinal logistic regression are more complicated, the fundamental approach is based on the same concepts as binary logistic regression, so that’s where we’ll focus today.

OK, so what are some examples of problems that might have a binary classification? Well, there are lots of them, including:

- Predicting whether a candidate will win an election.

- Predicting whether somebody will default on their loan.

- Predicting whether a tumor is cancerous.

- Predicting whether somebody will click on an advertisement.

- Predicting whether somebody will eat at Wendy’s in the next week.

### Probabilities and the Sigmoid Function

One thing we could try for our binary classification is a linear regression model with a threshold. We could
look at all our data and try to draw a line through it that minimizes the sum of squares error. Then, we
could determine a threshold value (say .5) such that if our linear model is above that threshold, then we
predict a positive outcome, and if it’s below we predict a negative outcome.

<center>
    <img src="Lin Regression.png" width="600">
</center>
    
This model might do... OK. At least when it comes to classification. However, there’s a real problem when
you want to interpret what the outcome means. If I’ve got a linear model that predictis how tall somebody
will be based on their parents’ height, and it comes back with 75 inches, that means their predicted height
is 75 inches. However, it’s not clear what an output of, say, .78 would mean in this case. Even worse, the
linear model can potentially have predicted values less than 0 or greater than 1. This is nuts! This would be a predicted value greater, or less, than the largest, or smallest, possible actual value. It’s very unclear what that should mean.

Well, what would we want the output value of our model to mean? These are all binary predictions, and
for almost any binary prediction, you’re probably interested in more than just a yes/no, true/false, 1/0
prediction. Most of the time, you want to not just know what the most likely outcome is, but you want to
have some idea of exactly how likely that outcome is. So, ideally, what we want from our logistic regression
model is a *probability*.

A probability is a number between 0 and 1, and so instead of a line, we’d want a function with a range from
0 to 1. A function that meets this requirement is the sigmoid, which is defined as:

<center>
$\displaystyle S(x) = \frac{1}{1+e^{-x}}$
</center>

And looks like:

<center>
    <img src="Sigmoid.png" width="600">
</center>

Now, there are very good mathematical reasons why the sigmoid function is what we use for logistic regression, but to get into that would divert us too much from the main concepts we want to cover. So, we won’t formally derive why we use the sigmoid,
but let’s explore it a bit to see how its properties align with what we’d want for a binary prediction model.

Some of the nice properties of the sigmoid include:

- Its values are constrained between 0 and 1, so no matter how large (positive) the input to the function
is, the output is always less than 1, and no matter how small (negative) the input to the function is,
the output is always greater than 0. That’s something we want if we’re going to treat these outputs as
probabilities.

- The sigmoid function is increasing, which means that if something makes an outcome more likely,
even more of that something makes the outcome even more likely. This makes intuitive sense, and
aligns with what we’d want for a straightforward probability model.

- The sigmoid function is smooth, which mathematically means it’s “differentiable", but more qualitatively means it doesn’t have any weird jumps or spikes in the probability, which is again what we’d want for a straightforward model.

- The sigmoid function has a region where the probabilities are very low, a region where the probabilities are very high, and a transition region where the probabilities are more in-between, which is what we’d expect intuitively for this type of classification problem.

### The Logistic Regression Model

OK, so now we’ve seen the sigmoid function. How do we use it to make a model? Where are the parameters? Well, for a logistic regression model, you actually start with a linear model:

&nbsp;

<center>
    $c_{1}X_{1} + c_{2}X_{2} + · · · + c_{n}X_{n} + b$
</center>

&nbsp;

and then you plug this linear model into the sigmoid function, which gives us:

&nbsp;

<center>
    $\displaystyle \frac{1}{1 + e^{−(c_{1}X_{1}+c_{2}X_{2}+···+c_{n}X_{n}+b)}}$
</center>

&nbsp;

If we’ve only got one independent variable, like in our model predicting the probability of passing the final
exam based on the number of hours studied, we have a univariate logistic regression model, and the formula
for our model (rewriting $c_{1}$ as $m$, and $X_{1}$ and $X$) is:

&nbsp;

<center>
    $\displaystyle \frac{1}{1 + e^{−(mX+b)}}$
</center>

&nbsp;

Now, as in linear regression, the task is to solve for the model that does the best job fitting our data. In
linear regression, this was the model that minimized the sum of squares between our predictions and our
data. However, in logistic regression, this sum of squares approach doesn’t make much sense. So, what do
we do?

Before we get into the answer, let’s step back and look at a different question. Suppose we have a
biased coin, which if you flip ends up heads with probability $p$, and tails with probability $(1 − p)$, and it is
not necessarily the case that $p = .5$. Suppose you flip the coin 10 times, and you get the following sequence
of heads and tails:

&nbsp;

<center>
    $H, H, T, H, T, H, H, H, T, H.$
</center>

&nbsp;

Now, for any value $0 < p < 1$, that sequence is technically possible. However, if the probability of heads
is $1\%$, so $p = .01$, then it’s very unlikely that you’d get 7 heads in 10 flips. On the other hand, if it were a
fair coin, so $p = .5$, then 7 heads in 10 flips is more likely. A natural question then is - for what value of
$p$ is the given sequence most likely? Formulating this mathematically, $p$ is the probability any given flip is
heads, $(1 − p)$ is the probability any given flip is tails, and the probability of any sequence of flips is just the
product of the probabilities of the individual flips, so:

&nbsp;

<center>
    $P(H, H, T, H, T, H, H, H, T, H) = pp(1 − p)p(1 − p)ppp(1 − p)p = p^{7}(1 − p)^{3}$
</center>

&nbsp;

This is the probability that, with ten flips of the coin, you’d see our sequence of heads and tails, given the
chance of heads is $p$. We call this the likelihood of our sequence, and the value of p that maximizes it is the
*maximum likelihood estimate*. In this example it’s a calculus exercise to show that the likelihood is maximized
at $p = .7$. In other words, if your coin shows up as heads $70\%$ of the time, your best guess is that its
probability of heads is $70\%$. Not exactly shocking.

Returning to logistic regression, suppose we’re looking at the first problem we discussed, predicting
whether a student will pass the final exam based on the hours they study, and we have the following
four data points:

| Student   | Hours of Study | Exam grade |
| :---:     |    :----:   | :---: |
| Student 1 | 3       | F |
| Student 2 | 2       | P |
| Student 3 | 0       | F |
| Student 4 | 4       | P |


Our equation for the probability of passing is:

&nbsp;

<center>
    $\displaystyle S(x, m, b) = \frac{1}{1 + e^{-(mX+b)}}$,
</center>

&nbsp;

where $X$ is the number of hours studied, and so the probability of the above four data points, the likelihood
of our data, will be given by:

&nbsp;
<center>
    $P(m, b) = (1 − S(3, m, b))S(2, m, b)(1 − S(0, m, b))S(4, m, b) = $
</center>
&nbsp;
<center>
    $\displaystyle \left(1 −\frac{1}{1+e^{-(3m+b)}}\right)\left(\frac{1}{1+e^{-(2m+b)}}\right)\left(1-\frac{1}{1+e^{-(0m+b)}}\right)\left(1+\frac{1}{e^{-(4m+b)}}\right)$
</center>
&nbsp;

Again, with linear regression, our goals was to find the values of m and b that minimized the sum of square
error. With logistic regression, our goal is to find the values of m and b that maximize the likelihood, which
for our four data points would be the function above. With more data points, it would be exactly the same
setup, just with more terms in our product.

Generally speaking, finding the values of the coefficients (in this case m and b) that maximize this likelihood function is hard. In our biased coin example, finding the exact value of p that gave us the maximum likelihood was possible. Unfortunately, that’s very rarely the case. Usually there’s no easy way to exactly calculate the maximum likelihood. 

Note that this is a big difference between logistic and linear regression. With linear regression, there are exact, optimized formulas that will tell you the coefficients that give you the best fit line. Not so with logistic regression.

If we can’t solve our equation exactly, what do we do? We use numerical methods to get close to the
exact solution. Precisely how this is done introduces a very important concept in data science and machine
learning. This very important concept is called gradient descent, which we'll discuss at the end of class today.

Alright, let's now dive into some code for doing logistic regression. First, we'll import some of the functions we'll need from sklearn.

In [ ]:
#Import the functions from sklearn we'll need
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
#Import the plotting function we'll use for data visualization
from matplotlib import pyplot as plt
import pandas as pd #Data manipulation and analysis library
import numpy as np #Numerical analysis library

First, we'll use the [make_classification](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html) function from sklearn to create a synthetic dataset that we can model.

In [ ]:
x, y = make_classification(

n_samples=300, #The number of samples in our dataset.
n_features=1, #The number of independent variables.
n_classes=2, #The number of possible outputs.
n_clusters_per_class=1, #The number of distinct data clusters in each class.
flip_y=0.05, #The fraction of samples whose values are assigned randomly.
n_informative=1, #The number of informative (independent) features.
n_redundant=0, #The number of features that can be derived.
n_repeated=0 #The number of repeated features.
)

We can then take a look at our synthetic dataset.

In [ ]:
plt.scatter(x, y, c=y, cmap='rainbow')
plt.title('Scatter Plot of Logistic Regression')
plt.show()

OK. We should be able to build a pretty decent regression model for this dataset. Let's split it into a training set and a test set.

In [ ]:
#Splits our data in training data and test data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.25)

Now, let's build our regression model.

In [ ]:
#Build and train the logistic regression model
log_reg = LogisticRegression()
log_reg.fit(x_train, y_train);

In [ ]:
#The coefficients for our logistic regression model
print(log_reg.coef_)
print(log_reg.intercept_)

We can check the prediction for the regression model for a given input - say 0.7.

In [ ]:
print(log_reg.predict([[0.7]])) #Our model's prediction for the given input

Let's graph our sigmoid function along with our data.

In [ ]:
#This will graph our sigmoid function along with our data.
# Sigmoid function
#
def sigmoid(z,m,b):
    return 1 / (1 + np.exp(-(m*z+b)))
# Creating sample Z points
#
z = np.arange(-5, 5, 0.1)
# Invoking Sigmoid function on all Z points
#
phi_z = sigmoid(z,log_reg.coef_[0][0],log_reg.intercept_[0])
# Plotting the Sigmoid function
#
plt.plot(z, phi_z)
plt.xlabel('z')
plt.ylabel('$\phi(z)$')
plt.yticks([0.0, 0.5, 1.0])
ax = plt.gca()
ax.yaxis.grid(True)
plt.tight_layout()
plt.scatter(x, y, c=y, cmap='rainbow')
plt.title('Scatter Plot of Logistic Regression')
plt.show()

Not bad!

### The Confusion Matrix

Once we have a logistic regression model, or really any type of classifier, an obvious question to ask is how
well it works. Very few models are perfect, and almost every one will get some classifications wrong. A
confusion matrix is a tabular way to record the successes and failures of a model, and provides a quick
overview of the model’s performance.

The rows of a confusion matrix represent the actual category classifications for our data, while the columns
represent the category classifications from our model. The entries represent the number of data elements
with the given actual and model classifications. For example, suppose we have a binary logistic regression
model predicting whether a tumor is malignant or benign, and it has the following confusion matrix:

| Tumor Diagnosis | Malignant | Benign |
| :---:     |    :----:   | :---: |
| Malignant | 29      | 2 |
| Benign    | 4       | 66 |

The first row of data represents the tumors that were actually malignant, and the second row of data rep-
resents the tumors that were actually benign. The first column of data represents the tumors the model classified as malignant, while the second column of data represents the tumors the model classified as benign.

From this we can see that there were 33 tumors classified as malignant, and of these 33, 29 were actually
malignant, while 4 were incorrectly classified, and were in reality benign. When a model marks a negative
(in this case benign) as a positive (in this case malignant) this is called a false positive or a type I error. We
can also see there were 68 tumors classified as benign, and of these 68, 66 were actually benign, while 2
were incorrectly classified, and were in reality malignant. When a model marks a positive (in this case
malignant) as a negative (in this case benign) this is called a false negative or a type II error.

Now, it’s not always the case that the cost associated with a false positive is the same as that associated
with a false negative. For example, if when the model says malignant the patient goes in for more tests,
then the cost of a false positive is relatively small compared to the cost of a false negative, which would
mean the patient doesn’t receive treatment when they should. In this case instead of setting the threshold
for a positive diagnosis at p > .5, you might set it at p > .05, which will significantly increase the number
of positive diagnoses, and increase the type I error (false positives) while decreasing the type II error (false
negatives). This might be fine given the nature of the problem. On the other hand, if when the model says
malignant the patient goes in for invasive surgery, then the cost of a false positive is significant, and we
wouldn’t bring the threshold down from p = .5 as much, if at all.

### Accuracy, Precision, Recall, and Specificity

There are a few terms associated with confusion matrices that any practicing data scientist needs to know.
These are:

- Accuracy - This one is pretty straightforward. The accuracy of a model is how often it’s right, regardless of whether it’s right about a true positive, or a true negative. The formula for accuracy is:

<center>
 $\text{Accuracy} = \displaystyle \frac{\text{Number of correct predictions}}{\text{Total number of predictions}}$
</center>

- Precision - The precision is concerned with the positive. Specifically, it is the fraction of the time the
model’s positive result is accurate. Defined precisely, the formula for precision is:

<center>
    $\text{Precision} = \displaystyle \frac{\text{True Positives}}{\text{Predicted Positives}} = \frac{\text{True Positives}}{\text{True Positives + False Positives}}$
</center>

- Recall - The recall is concerned with how often the model correctly identifies an actual positive. Its formula is:

<center>
    $\text{Recall} = \displaystyle \frac{\text{True Positives}}{\text{Real Positives}} = \frac{\text{True Positives}}{\text{True Positives + False Negatives}}$
</center>

- Specificity - The specificity is concerned with how often the model correctly identifies an actual negative. Its formula is:

<center>
    $\text{Specificity} = \displaystyle \frac{\text{True Negatives}}{\text{Real Negatives}} = \frac{\text{True Negatives}}{\text{True Negatives + False Positives}}$
</center>

In a great model, ideally, all of these will be high. However, in reality, frequently tradeoffs must be made,
and which of these you choose to optimize will depend on the nature of your problem, and the different
costs for different types of mistakes. For example, in our tumor example, which of these do you think
would be most important?

Everything we’ve talked about so far can be applied to any binary classification problem, and it’s straight-
forward to generalize these ideas to non-binary classification as well. However, remember, one of the big ideas behind logistic regression was that it not only gave you a yes/no prediction, but it also gave you a probability. Is there a way of measuring how good those probability scores are? Yes, there is!

### The ROC Curve and AUC

The ROC curve (which stands for the receiver operating characteristic curve) is a graph showing the performance of a classification model at all classification thresholds. The curve plots two parameters, the true positive rate (TPR), which is the recall, on the y-axis, and the false positive rate (FPR), which is 1 minus the specificity, on the x-axis. When the threshold is p = 1, so as strict as it could be, then nothing will be classified as positive, and both the TPR and FPR will be 0. When the threshold is p = 0, so as relaxed as it could be, then everything will be classified as positive, and both the TPR and FPR will be 1. As we move from p = 1 down to p = 0, the TPR and FPR will grow at their own rates, and we can plot their values on a curve. This is the ROC curve.

<center>
    <img src="ROC.png" width="600">
</center>

We measure the performance of the model by looking at the area under the ROC curve. This performance
measure is called AUC, which just stands for "Area Under the Curve". This is a number between 0 and 1,
and would be 1 for a perfect model (one that gave every real positive probability 1, and every real negative
probability 0). For a random model, you’d get an AUC of about .5. A good AUC is usually between .8 and
.9.

## Gradient Descent

Let's return to the question of how we can find the maximum likelihood estimate for a logistic regression.

The function that we use for predictions in logistic regression is the sigmoid function:

&nbsp;

<center>
    $\displaystyle S(\textbf{X}) = \frac{1}{1 + e^{−(c_{1}X_{1}+c_{2}X_{2}+···+c_{n}X_{n}+b)}}$
</center>
&nbsp;

If we're given a set of input data and we want to find the best predictor, we don't try to minimize the sum of squares error as we did with linear regression. Instead, with logistic regression, we try to *maximize the likelihood*. In other words, we try to find the values of our coefficients such that the probability of our dataset is maximized.

Now, unfortunately, the exact values that achieve this aren't always easy to find. For the sigmoid function and logistic regression, it's not really possible to find a closed-form solution the way we can with linear regression. So, to get around this difficulty, we use *gradient descent*. 

The basic idea is suppose you’re on a mountain and you’re trying to get down it as quickly as possible, but you’re in a fog and so you can only see a couple feet in front of you. You
don’t know where the lowest point on the mountain is, but you do know the direction that will get you
down the farthest on your next step. So, you just take a step in that direction, look around, figure out the
step that will take you down the farthest from your new position, take that step, and so on. Every time,
you’re taking the step that gets you down the farthest locally, and hopefully this approach will get you to
the bottom of the mountain ASAP.

&nbsp;

<center>
    <img src="Gradient_Descent.gif" width="600">
</center>

&nbsp;

Stated more mathematically, at any given point it's frequently straightforward to find the gradient (the vector of partial derivatives) of our function. This gradient tells us the direction of greatest increase at that point, and so if we're looking to maximize a function, we can take a step in that direction. If we're looking to minimize the function, we can take a step in the other direction.

For the sigmoid function, the partial derivative with respect to a coefficient is relatively easy to calculate:

&nbsp;

<center>
    $\displaystyle \frac{\partial S}{\partial c_{i}} = \frac{X_{i}e^{−(c_{1}X_{1}+c_{2}X_{2}+···+c_{n}X_{n}+b)}}{\left(1 + e^{−(c_{1}X_{1}+c_{2}X_{2}+···+c_{n}X_{n}+b)}\right)^{2}} = X_{i}\left(1-\frac{1}{1 + e^{-(c_{1}X_{1}+c_{2}X_{2}+···+c_{n}X_{n}+b)}}\right)\left(\frac{1}{1 + e^{−(c_{1}X_{1}+c_{2}X_{2}+···+c_{n}X_{n}+b)}}\right) = X_{i}(1-S)S$
</center>
&nbsp;

Here $c_{0} = b$ and $X_{0} = 1$.

From this, we can fairly easily calculate the gradient at any point $\textbf{X}$. Now, an important hyperparameter here is the step size - how far you move in the direction of (or opposite) the gradient at each step. This is sometimes knows as the *learning rate*, and its study is an important field within machine learning. For today, we'll assume the step size is constant, but please note that isn't always the case.

Something to note is that the product of terms in our likelihood is equal to the number of observations in our dataset. For a relatively small dataset this isn't a problem, but for a huge dataset this can make likelihood calculations rather involved, and iterative likelihood calculations extremely resource intensive.

#### Stochastic Gradient Descent

One way to get around this issue is with *stochastic gradient descent*. The idea behind stochastic gradient descent is that (in the extreme) we only see how our prediction works on one single observation, and then we adjust our parameters accordingly based upon our prediction for that observation. In other words, we move either forward or backward depending on whether our prediction was right or wrong.

This can make the optimization problem *much* less computationally intensive, although it does introduce some potential problems, and makes your optimization dependent on the order in which you evaluate the observations. A middle ground between pure gradient descent and extreme stochastic gradient descent is batched stochastic gradient descent, where we divide our data into disjoint groups, and run gradient descent over each group individually.

Let's take a look at how we could implement logistic regression with stochastic gradient descent. In today's lecture, instead of relying on our standard libraries like pandas and numpy, we're going to try something different and (as much as possible) write everything from scratch, just to see how it would be done.

First, let's take a look at a simple sample dataset with two predictive inputs and a binary categorical output.

In [ ]:
# Sample Data
dataset = [[2.7810836,2.550537003,0],
[1.465489372,2.362125076,0],
[3.396561688,4.400293529,0],
[1.38807019,1.850220317,0],
[3.06407232,3.005305973,0],
[7.627531214,2.759262235,1],
[5.332441248,2.088626775,1],
[6.922596716,1.77106367,1],
[8.675418651,-0.242068655,1],
[7.673756466,3.508563011,1]]

For a set of observations and a set of coefficients for our logistic regression model, we can write a function that makes a prediction (0 or 1) for each observation.

In [ ]:
from math import exp

# Make a prediction with coefficients
def predict(row, coefficients):
    X = coefficients[0]
    for i in range(len(row)-1):
        X += coefficients[i + 1] * row[i]
    return 1.0 / (1.0 + exp(-X))

Let's try this out with some initial coefficients all set to $1.0$.

In [ ]:
coef = [1.0, 1.0, 1.0]

In [ ]:
for row in dataset:
    yhat = predict(row, coef)
    print("Expected=%.3f, Predicted=%.3f [%d]" % (row[-1], yhat, round(yhat)))

Pretty much a zero-shot always predict 1 model. Let's see if we can use stochastic gradient descent to do better.

First, let's write a function for performing stochastic gradient descent.

In [ ]:
# Estimate logistic regression coefficients using stochastic gradient descent
def coefficients_sgd(train, l_rate, n_epoch):
    coef = [1.0 for i in range(len(train[0]))] #Start everything at 0.0
    for epoch in range(n_epoch):
        for row in train:
            yhat = predict(row, coef)
            error = row[-1] - yhat
            coef[0] = coef[0] + l_rate * error * yhat * (1.0 - yhat)
            for i in range(len(row)-1):
                coef[i + 1] = coef[i + 1] + l_rate * error * yhat * (1.0 - yhat) * row[i]
    return coef

Then, let's give it a run with a learning rate of $.3$, and 100 epochs (trips through the dataset).

In [ ]:
l_rate = 0.3
n_epoch = 100
coef = coefficients_sgd(dataset, l_rate, n_epoch)
print(coef)

Hopefully, this improves the predictive power of our model.

In [ ]:
for row in dataset:
    yhat = predict(row, coef)
    print("Expected=%.3f, Predicted=%.3f [%d]" % (row[-1], yhat, round(yhat)))

Nailed it! In homework assignment #4, you'll try something a bit more involved.